In [15]:
import math
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
import pandas as pd
import numpy as np
from scipy.stats import zscore
from itemchm.persistence.entities import UserBook

from itemchm.utils import overall_rate

ENGINE = create_engine("sqlite:///bookshelf.db", echo=True)

In [10]:
user_info = {}
book_info = {}

with Session(ENGINE) as session:
    user_books = session.query(UserBook).all()

for user_book in user_books:
    user_id = user_book.userId
    book_id = user_book.bookId

    index = math.floor(overall_rate(user_book)) - 1

    if user_id not in user_info:
        user_info[user_id] = np.zeros(6)

    if book_id not in book_info:
        book_info[book_id] = np.zeros(6) 

    user_info[user_id][index] += 1 
    book_info[book_id][index] += 1


2024-08-31 21:27:02,682 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-31 21:27:02,684 INFO sqlalchemy.engine.Engine SELECT "UserBooks".id AS "UserBooks_id", "UserBooks"."readRatio" AS "UserBooks_readRatio", "UserBooks".rating AS "UserBooks_rating", "UserBooks".comment AS "UserBooks_comment", "UserBooks".shared AS "UserBooks_shared", "UserBooks"."bookId" AS "UserBooks_bookId", "UserBooks"."userId" AS "UserBooks_userId" 
FROM "UserBooks"
2024-08-31 21:27:02,684 INFO sqlalchemy.engine.Engine [generated in 0.00057s] ()
2024-08-31 21:27:03,317 INFO sqlalchemy.engine.Engine ROLLBACK


In [32]:
# Example user ratings dictionary
user_ratings = {1: [24, 0, 1, 2, 34, 25], 2: [10, 5, 8, 12, 15, 20]}

# Convert to DataFrame
user_df = pd.DataFrame.from_dict(user_info, orient='index', columns=[1, 2, 3, 4, 5, 6])

# Summary statistics

# Total Ratings
user_df['Total Ratings'] = user_df.sum(axis=1)

# Average Rating
ratings_columns = [1, 2, 3, 4, 5, 6]
user_df['Average Rating'] = (user_df[ratings_columns] * ratings_columns).sum(axis=1) / user_df['Total Ratings']

# Most Frequent Rating
user_df['Most Frequent Rating'] = user_df[ratings_columns].idxmax(axis=1)

user_df.describe()

,1,2,3,4,5,6,Total Ratings,Average Rating
count,499.000000,499.000000,499.0,499.000000,499.000000,499.000000,499.000000,499.000000
mean,70.200401,22.727455,0.0,11.046092,31.681363,0.144289,135.799599,2.333549
std,55.021645,18.112485,0.0,15.560733,43.702130,0.438167,74.837897,1.199527
min,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,1.000000
25%,24.000000,7.000000,0.0,1.000000,4.000000,0.000000,72.000000,1.402908
50%,49.000000,16.000000,0.0,3.000000,8.000000,0.000000,141.000000,1.504950
75%,116.500000,39.000000,0.0,15.500000,48.500000,0.000000,199.000000,3.854670
max,200.000000,69.000000,0.0,64.000000,157.000000,3.000000,271.000000,4.666667


In [44]:
# Convert to DataFrame
user_df = pd.DataFrame.from_dict(user_info, orient='index', columns=[1, 2, 3, 4, 5, 6])

ratings_series = user_df.values.flatten()

# Calculate Z-scores
z_scores = zscore(ratings_series)

# Define a threshold for anomaly detection
threshold = 4

# Identify anomalies
anomalies = np.abs(z_scores) > threshold

# Reshape the result to match the original DataFrame's structure
anomalies_reshaped = anomalies.reshape(user_df.shape)

# Create a DataFrame to display anomalies
anomalies_df = pd.DataFrame(anomalies_reshaped, index=user_df.index, columns=user_df.columns)

In [45]:
anomalies_total = anomalies_df.sum().sum()
total = anomalies_df.shape[0] * anomalies_df.shape[1]
anomalies_percentage = (anomalies_total / total) * 100
anomalies_percentage

0.36740146960587844

In [40]:
# Example user ratings dictionary
user_ratings = {1: [24, 0, 1, 2, 34, 25], 2: [10, 5, 8, 12, 15, 20]}

# Convert to DataFrame
user_df = pd.DataFrame.from_dict(book_info, orient='index', columns=[1, 2, 3, 4, 5, 6])

# Summary statistics

# Total Ratings
user_df['Total Ratings'] = user_df.sum(axis=1)

# Average Rating
ratings_columns = [1, 2, 3, 4, 5, 6]
user_df['Average Rating'] = (user_df[ratings_columns] * ratings_columns).sum(axis=1) / user_df['Total Ratings']

# Most Frequent Rating
user_df['Most Frequent Rating'] = user_df[ratings_columns].idxmax(axis=1)

user_df.describe()

,1,2,3,4,5,6,Total Ratings,Average Rating
count,1000.000000,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,35.030000,11.341000,0.0,5.512000,15.809000,0.072000,67.764000,2.346321
std,8.546235,3.947696,0.0,3.213438,7.665982,0.266246,7.386262,0.500285
min,16.000000,2.000000,0.0,0.000000,0.000000,0.000000,48.000000,1.109375
25%,29.000000,8.000000,0.0,3.000000,14.000000,0.000000,63.000000,2.302679
50%,33.000000,11.000000,0.0,6.000000,18.000000,0.000000,68.000000,2.513889
75%,39.000000,14.000000,0.0,8.000000,21.000000,0.000000,72.250000,2.652778
max,68.000000,28.000000,0.0,17.000000,31.000000,2.000000,92.000000,3.116667


In [48]:
# Convert to DataFrame
user_df = pd.DataFrame.from_dict(book_info, orient='index', columns=[1, 2, 3, 4, 5, 6])

ratings_series = user_df.values.flatten()

# Calculate Z-scores
z_scores = zscore(ratings_series)

# Define a threshold for anomaly detection
threshold = 4

# Identify anomalies
anomalies = np.abs(z_scores) > threshold

# Reshape the result to match the original DataFrame's structure
anomalies_reshaped = anomalies.reshape(user_df.shape)

# Create a DataFrame to display anomalies
anomalies_df = pd.DataFrame(anomalies_reshaped, index=user_df.index, columns=user_df.columns)

In [49]:
anomalies_total = anomalies_df.sum().sum()
total = anomalies_df.shape[0] * anomalies_df.shape[1]
anomalies_percentage = (anomalies_total / total) * 100
anomalies_percentage

0.03333333333333333